In [1]:
# Work Cited:
# Dash Data table https://dash.plotly.com/datatable
# Range Slider https://dash.plotly.com/dash-core-components/rangeslider
# Pie Charts in Python https://plotly.com/python/pie-charts/
# Mongo DB Manual https://www.mongodb.com/docs/manual/
# Dash Leaflet https://www.dash-leaflet.com/docs/getting_started

from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_leaflet as dl

import plotly.express as px
import pandas as pd
import numpy as np
import base64, os, math

# our CRUD class for MongoDB
from CRUD import AnimalShelter

# ---------- setup / data ----------
username = "aacuser1"
password = "SNHU1234"
CSV_PATH = "aac_shelter_outcomes.csv" 

db = AnimalShelter(username, password)

def load_all_data():
    # try DB first
    try:
        df = pd.DataFrame.from_records(db.read({}))
    except Exception:
        df = pd.DataFrame()
    if df.empty and os.path.exists(CSV_PATH):
        df = pd.read_csv(CSV_PATH)
    # clean up
    if "_id" in df.columns:
        df = df.drop(columns=["_id"])
    if "age_upon_outcome_in_weeks" in df.columns:
        df["age_upon_outcome_in_weeks"] = pd.to_numeric(df["age_upon_outcome_in_weeks"], errors="coerce")
        df["age_years"] = df["age_upon_outcome_in_weeks"] / 52.0
    else:
        df["age_years"] = pd.Series(dtype=float)
    return df

DF_ALL = load_all_data()

# dropdown/checklist options
ALL_BREEDS = sorted([b for b in DF_ALL.get("breed", pd.Series()).dropna().unique().tolist()][:500])
ALL_SEX = ["Intact Male", "Intact Female", "Neutered Male", "Spayed Female", "Unknown"]

# slider bounds 
age_years = DF_ALL.get("age_years", pd.Series(dtype=float)).dropna()
AGE_Y_MIN = float(np.floor(age_years.min())) if not age_years.empty else 0.0
AGE_Y_MAX = float(np.ceil(age_years.max()))  if not age_years.empty else 3.0

AGE_MARKS = {int(y): str(int(y)) for y in range(int(AGE_Y_MIN), int(AGE_Y_MAX) + 1)}

# default filter values
DEFAULTS = {
    "category": "All",
    "breeds": [],
    "sex": [],
    "age_years": [AGE_Y_MIN, AGE_Y_MAX],
}

# category filters
CATEGORY_QUERIES = {
    "water": {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "NewFoundLand"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0},
    },
    "mount": {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0},
    },
    "disaster": {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0},
    },
}

def years_to_weeks_range(age_years_range):
    # convert slider years to weeks for Mongo
    if not age_years_range or len(age_years_range) != 2:
        return None
    lo_w = float(age_years_range[0]) * 52.0
    hi_w = float(age_years_range[1]) * 52.0
    if lo_w > hi_w:
        lo_w, hi_w = hi_w, lo_w
    return {"$gte": lo_w, "$lte": hi_w}

def build_query(category, breeds, sex_list, age_years_range):
    # put the filters together for Mongo
    q = {}
    if category in CATEGORY_QUERIES:
        q.update(CATEGORY_QUERIES[category])
    weeks_cond = years_to_weeks_range(age_years_range)
    if weeks_cond:
        q["age_upon_outcome_in_weeks"] = weeks_cond
    if breeds:
        q["breed"] = {"$in": breeds}
    if sex_list:
        q["sex_upon_outcome"] = {"$in": sex_list}
    return q

def fetch_df(category, breeds, sex_list, age_years_range):
    # try Mongo with our query
    q = build_query(category, breeds, sex_list, age_years_range)
    try:
        data = db.read(q)
        df = pd.DataFrame.from_records(data)
        if "_id" in df.columns:
            df.drop(columns=["_id"], inplace=True)
        if "age_upon_outcome_in_weeks" in df.columns:
            df["age_years"] = pd.to_numeric(df["age_upon_outcome_in_weeks"], errors="coerce") / 52.0
    except Exception:
        df = DF_ALL.copy()
        if category in CATEGORY_QUERIES:
            cq = CATEGORY_QUERIES[category]
            if "animal_type" in cq and "animal_type" in df:
                df = df[df["animal_type"] == cq["animal_type"]]
            if "breed" in cq and "breed" in df and "$in" in cq["breed"]:
                df = df[df["breed"].isin(cq["breed"]["$in"])]
            if "sex_upon_outcome" in cq and "sex_upon_outcome" in df:
                df = df[df["sex_upon_outcome"] == cq["sex_upon_outcome"]]
            if "age_upon_outcome_in_weeks" in cq and "age_upon_outcome_in_weeks" in df:
                lo = cq["age_upon_outcome_in_weeks"]["$gte"]
                hi = cq["age_upon_outcome_in_weeks"]["$lte"]
                df = df[(df["age_upon_outcome_in_weeks"] >= lo) & (df["age_upon_outcome_in_weeks"] <= hi)]
        if "age_years" in df and age_years_range and len(age_years_range) == 2:
            ylo, yhi = float(age_years_range[0]), float(age_years_range[1])
            if ylo > yhi:
                ylo, yhi = yhi, ylo
            df = df[(df["age_years"] >= ylo) & (df["age_years"] <= yhi)]
        if breeds and "breed" in df:
            df = df[df["breed"].isin(breeds)]
        if sex_list and "sex_upon_outcome" in df:
            df = df[df["sex_upon_outcome"].isin(sex_list)]
    return df

# ---------- app / layout ----------
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True
app.config.prevent_initial_callbacks = True

# logo
logo_src = None
logo_path = "Grazioso Salvare Logo.png"
if os.path.exists(logo_path):
    with open(logo_path, "rb") as f:
        logo_src = f"data:image/png;base64,{base64.b64encode(f.read()).decode()}"

app.layout = html.Div([
    html.Center(html.B(html.H1("Grazioso Salvare"))),
    html.Center(
        html.Img(src=logo_src, style={"width": "400px", "height": "auto"})
        if logo_src else html.Div("(Logo not found)")
    ),
    html.Center(html.H4("Brett Nottmeier")),
    html.Hr(),

    # keep defaults in memory for reset button
    dcc.Store(id="defaults-store", data=DEFAULTS),

    # 4 column filter 
    html.Div(
        style={
            "display": "grid",
            "gridTemplateColumns": "repeat(4, 1fr)",
            "gap": "12px",
            "alignItems": "start",
        },
        children=[
            # 1) category
            html.Div([
                html.Label("Rescue Category"),
                dcc.RadioItems(
                    id="filter-options",
                    options=[
                        {"label": "Water Rescue", "value": "water"},
                        {"label": "Mountain/Wilderness Rescue", "value": "mount"},
                        {"label": "Disaster/Individual Tracking", "value": "disaster"},
                        {"label": "All Rescues", "value": "All"},
                    ],
                    value=DEFAULTS["category"],
                    labelStyle={"display": "block"},
                ),
            ]),
            # 2) breeds
            html.Div([
                html.Label("Breeds (you can pick many)"),
                dcc.Dropdown(
                    id="breed-dropdown",
                    options=[{"label": b, "value": b} for b in ALL_BREEDS],
                    value=DEFAULTS["breeds"],
                    multi=True,
                    placeholder="Pick one or more breeds",
                ),
            ]),
            # 3) sex
            html.Div([
                html.Label("Sex"),
                dcc.Checklist(
                    id="sex-checklist",
                    options=[{"label": s, "value": s} for s in ALL_SEX],
                    value=DEFAULTS["sex"],
                ),
            ]),
            # 4) age years
            html.Div([
                html.Label("Age (years)"),
                dcc.RangeSlider(
                    id="age-slider",
                    min=AGE_Y_MIN,
                    max=AGE_Y_MAX,
                    step=0.5,
                    value=DEFAULTS["age_years"],
                    marks=AGE_MARKS,
                    tooltip={"placement": "bottom", "always_visible": True},
                ),
                html.Div(id="age-readout", style={"marginTop": "6px", "textAlign": "center"}),
            ]),
        ],
    ),

    # reset button
    html.Div(style={"marginTop": "10px"}, children=[
        html.Button("Reset Filters", id="reset-button", n_clicks=0)
    ]),

    html.Hr(),

    # table
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in DF_ALL.columns],
        data=DF_ALL.to_dict("records"),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        page_action="native",
        page_current=0,
        page_size=10,
        style_table={"overflowX": "auto"},
    ),

    html.Br(), html.Hr(),

    # two panels
    html.Div(className="row", style={"display": "flex", "gap": "12px"}, children=[
        html.Div(id="graph-id", className="col s12 m6", style={"flex": 1}),
        html.Div(id="map-id",   className="col s12 m6", style={"flex": 1}),
    ])
])

# ---------- callbacks ----------

# show the age slider range as text
@app.callback(Output("age-readout", "children"), Input("age-slider", "value"))
def _age_readout(age_years):
    if age_years and len(age_years) == 2:
        a, b = float(age_years[0]), float(age_years[1])
        def fmt(x): return f"{int(x)}" if float(x).is_integer() else f"{x:.1f}"
        return f"{fmt(a)} – {fmt(b)} years"
    return ""

# reset all filters back to defaults
@app.callback(
    Output("filter-options", "value"),
    Output("breed-dropdown", "value"),
    Output("sex-checklist", "value"),
    Output("age-slider", "value"),
    Input("reset-button", "n_clicks"),
    State("defaults-store", "data"),
    prevent_initial_call=True,
)
def _reset(n, defaults):
    if n:
        return defaults["category"], defaults["breeds"], defaults["sex"], defaults["age_years"]
    raise PreventUpdate

# when filters change, update the table data
@app.callback(
    Output("datatable-id", "data"),
    Input("filter-options", "value"),
    Input("breed-dropdown", "value"),
    Input("sex-checklist", "value"),
    Input("age-slider", "value"),
)
def _update_table(category, breeds, sex_list, age_years_range):
    df = fetch_df(category, breeds or [], sex_list or [], age_years_range or DEFAULTS["age_years"])
    return df.to_dict("records")

# make a pie chart of breeds based on whats in the table
@app.callback(Output("graph-id", "children"), Input("datatable-id", "derived_virtual_data"))
def _update_graph(viewData):
    if not viewData:
        return html.Div("No data to display")
    dff = pd.DataFrame(viewData)
    if "breed" not in dff.columns or dff.empty:
        return html.Div("No breed data available")
    counts = dff["breed"].value_counts().reset_index()
    counts.columns = ["breed", "count"]
    top = counts.head(12)
    other_sum = counts["count"][12:].sum()
    if other_sum:
        top = pd.concat([top, pd.DataFrame([{"breed": "Other", "count": other_sum}])], ignore_index=True)
    fig = px.pie(top, names="breed", values="count", title="Preferred Dog Breeds (filtered)")
    return [dcc.Graph(figure=fig)]

# update the map when a row is selected default: first row
@app.callback(
    Output("map-id", "children"),
    Input("datatable-id", "derived_virtual_data"),
    Input("datatable-id", "derived_virtual_selected_rows"),
)
def _update_map(viewData, selected_rows):
    if not viewData:
        return html.Div("No data to display")
    dff = pd.DataFrame(viewData)
    row = (selected_rows or [0])[0]
    if row is None or row >= len(dff):
        return html.Div("Pick a row in the table to see its location")
    if "location_lat" not in dff.columns or "location_long" not in dff.columns:
        return html.Div("No location data available")

    try:
        lat = float(dff.iloc[row]["location_lat"])
        lon = float(dff.iloc[row]["location_long"])
    except Exception:
        return html.Div("Invalid location values")

    name  = dff.iloc[row].get("name", "Unknown")
    breed = dff.iloc[row].get("breed", "Unknown")

    return [
        dl.Map(style={"width": "100%", "height": "500px"}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H4("Animal"), html.P(f"Name: {name}"), html.P(f"Breed: {breed}")])
            ])
        ])
    ]

# ---------- run ----------
if __name__ == "__main__":
    app.run_server(host="127.0.0.1", port=8050, debug=False)



/home/brett/repos/artifact three/.venv/lib/python3.12/site-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
